# DevNet - API - Presentation 05

#### **Task:** Provide API calls to obtain list of clients/hosts on a network using Meraki or DNA

#### **Solution:** 
I have prepared python script that is listing all clients through out whole hierarchy (organisations -> networks -> devices -> clients)

#### **Output:**
- output.json -> here is whole hierarchy presented (clients per device per network per organisation)
- all_clients_output.json -> here are extracted all clients

**Note:** Output can be further analysed for better organisation and potential deduplication|

#### Requirements:
***config*** file with Meraki API key should be added in the working directory
</br>Format:
```json
    {"meraki_api_key" : "__API_KEY__"}
```

In [ ]:
#!/usr/bin/env python3

import os
import sys
import time
import json

import requests

#start = time.time()

# GET CONFIGURATION
conf_file = 'config'
if os.path.exists(conf_file):
  config = json.loads(open(conf_file).read())
else:
  sys.exit(2)

# BASE PARAMETERS
merkai_base_url = 'https://api.meraki.com/api/v1'
meraki_api_key = config['meraki_api_key']
header = {'X-Cisco-Meraki-API-Key':meraki_api_key}

# RESULT
organisations = []
networks = []
clients = []
devices = []
all_clients = []

# LIST ALL ORGANISATIONS
list_organisations_ep = '/organizations'
try:
  org_req = requests.get(f'{merkai_base_url}{list_organisations_ep}', headers=header)
  org_data = org_req.json()

  if org_data != None:
    for org in org_data:
      if org != None:
        org_id = org['id']
        org_name = org['name']

        #print(org_name, org_id)

        # LIST ALL NETWORKS FOR EACH ORGANISATION
        list_networks_ep = f'/organizations/{org_id}/networks'
        try:
          network_req = requests.get(f'{merkai_base_url}{list_networks_ep}', headers=header)
          network_data = network_req.json()
          
          if network_data != None:
            for network in network_data:
              if network != None:
                network_id = network['id']
                network_name = network['name']

                #print(network_name, network_id)

                #LIST ALL DEVICES FOR EACH NETWORK
                list_deviced_ep = f'/networks/{network_id}/devices'
                try:
                  devices_req = requests.get(f'{merkai_base_url}{list_deviced_ep}', headers=header)
                  devices_data = devices_req.json()
                  
                  if devices_data != None:
                    for device in devices_data:
                      if device != None:
                        device_serial = device['serial']
                        if 'name' in device:
                          device_name = device['name']
                        else:
                          device_name = 'unknown' 

                        #print(device_name, device_serial)

                        #LIST ALL CLIENTS FOR EACH DEVICE
                        list_all_clients_ep = f'/devices/{device_serial}/clients'
                        try:
                          clients_req = requests.get(f'{merkai_base_url}{list_all_clients_ep}', headers=header)
                          clients_data = clients_req.json()
                          
                          if clients_data != None:
                            for client in clients_data:
                              if client != None:
                                client_id = client['id']
                                client_mac = client['mac']
                                client_description = 'none'
                                if 'description' in  client:
                                  client_description = client['description']
                                clients.append({'id':client_id, 'mac':client_mac, 'description':client_description})
                                all_clients.append({'id':client_id, 'mac':client_mac, 'description':client_description})

                                #print(client_id, client_mac, client_description)
                        except:
                          print('Hiccup while listing all clients', clients_req.status_code)

                        devices.append({'serial':device_serial, 'name':device_name, 'clients':clients})
                        clients = []
                except:
                  print('Hiccup while listing all devices', devices_req.status_code)  

                networks.append({'id':network_id, 'name':network_name, 'devices':devices})
                devices = []    
        except:
          print('Hiccup while listing all networks', network_req.status_code)

        organisations.append({'id':org_id, 'name':org_name, 'networks':networks})
        networks = []
except:
  print('Hiccup while listing all organisations', org_req.status_code)

with open ('output.json', 'w') as otp:
  json.dump(organisations, otp)

with open ('all_clients_output.json', 'w') as otp:
  json.dump(all_clients, otp)

#end = time.time()
#print(f'Execution time: {end-start}')

Hiccup while listing all clients 400
Hiccup while listing all clients 400
